# Modelo de regresión logística

# Programa o Script desarrollado por : Rolando Ortiz Herbas

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from decimal import Decimal, getcontext
import csv

In [2]:
ruta = "C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/OUTPUT_PCA.csv"
datasetT = pd.read_csv(ruta , header=0)

In [3]:
df = pd.DataFrame(datasetT)

# Tomar el 80% de los datos de manera aleatoria para entrenamiento
df_entrenamiento = df.sample(frac=0.8, random_state=1)

# Usar el 20% restante para pruebas
df_prueba = df.drop(df_entrenamiento.index)

In [4]:
df_entrenamiento.to_csv('C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/OUTPUT_PCA_Entrenamiento.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [5]:
df_prueba.to_csv('C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/OUTPUT_PCA_Prueba.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

# Lectura del Dataset de entrenamiento : Clientes_Dos_Mil_TransEntrenamiento.csv

In [6]:
getcontext().prec = 30
ruta = "C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/OUTPUT_PCA_Entrenamiento.csv"

df = pd.read_csv(ruta , header=0)

In [7]:
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,Y
0,-1.753884,-0.958379,-1.650878,-0.742405,-1.262931,-1.654056,-0.207998,-1.422868,-1.731674,0.978161,0.528280,0.267227,0.460100,-0.680924,-0.363155,1
1,0.888191,-1.557781,-1.527159,-0.254174,-0.910098,-0.865243,-0.602647,-1.184309,-1.404302,0.780190,0.579684,0.706622,-0.073110,0.576587,-0.349935,1
2,5.567092,-2.238378,0.390915,-0.929111,0.978921,-1.307864,-0.186563,0.314617,1.382592,-0.131808,-0.134406,-0.950579,-0.158812,-0.074169,0.198908,1
3,0.261184,-1.811012,1.533936,-1.172504,2.229075,0.357651,0.895314,0.193314,0.306810,0.142421,-0.065682,0.197883,-0.668622,-0.886527,0.396419,1
4,6.990773,1.381154,0.847240,-0.845116,-1.253855,1.392332,0.915926,-0.023725,-2.229061,1.260231,0.704750,-0.813934,0.171559,-0.943656,0.416792,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     1600 non-null   float64
 1   PC2     1600 non-null   float64
 2   PC3     1600 non-null   float64
 3   PC4     1600 non-null   float64
 4   PC5     1600 non-null   float64
 5   PC6     1600 non-null   float64
 6   PC7     1600 non-null   float64
 7   PC8     1600 non-null   float64
 8   PC9     1600 non-null   float64
 9   PC10    1600 non-null   float64
 10  PC11    1600 non-null   float64
 11  PC12    1600 non-null   float64
 12  PC13    1600 non-null   float64
 13  PC14    1600 non-null   float64
 14  PC15    1600 non-null   float64
 15  Y       1600 non-null   int64  
dtypes: float64(15), int64(1)
memory usage: 200.1 KB


In [9]:
df = df.astype({col: 'float64' for col in df.select_dtypes(include='int64').columns})

# Verificar los tipos de datos
print(df.dtypes)

PC1     float64
PC2     float64
PC3     float64
PC4     float64
PC5     float64
PC6     float64
PC7     float64
PC8     float64
PC9     float64
PC10    float64
PC11    float64
PC12    float64
PC13    float64
PC14    float64
PC15    float64
Y       float64
dtype: object


# Inicializa coeficientes y lee las variables

In [10]:


B = pd.Series([0] * 30)
B0 = pd.Series([0] * 30)


X = np.array(df.iloc[:, :-1])

X1 = df.iloc[:, :-1]
#X = np.array(X1)
Y = np.array(df["Y"])


In [11]:
X1.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
0,-1.753884,-0.958379,-1.650878,-0.742405,-1.262931,-1.654056,-0.207998,-1.422868,-1.731674,0.978161,0.528280,0.267227,0.460100,-0.680924,-0.363155
1,0.888191,-1.557781,-1.527159,-0.254174,-0.910098,-0.865243,-0.602647,-1.184309,-1.404302,0.780190,0.579684,0.706622,-0.073110,0.576587,-0.349935
2,5.567092,-2.238378,0.390915,-0.929111,0.978921,-1.307864,-0.186563,0.314617,1.382592,-0.131808,-0.134406,-0.950579,-0.158812,-0.074169,0.198908
3,0.261184,-1.811012,1.533936,-1.172504,2.229075,0.357651,0.895314,0.193314,0.306810,0.142421,-0.065682,0.197883,-0.668622,-0.886527,0.396419
4,6.990773,1.381154,0.847240,-0.845116,-1.253855,1.392332,0.915926,-0.023725,-2.229061,1.260231,0.704750,-0.813934,0.171559,-0.943656,0.416792


In [12]:
alfa = 0.3
n = X.shape[1]
Nfilas = X.shape[0]


In [13]:

out = 0.0
for i in range(n+1):
    
    if i == 0 :
        out = B[0]
    else:
        out = out + B[i] * X[0,i-1]   
pred = 0.0        
pred = 1/(1 + math.exp(-out)) 



In [14]:
for i in range(n+1):
            
    if i == 0:
        B[i] = B[i] + alfa * (Y[0]- pred)*1
    else:
        B[i] = B[i] + alfa * (Y[0]- pred)*X[0,i-1]
               
 

# Inicia el aprendizaje

In [15]:
for k in range(10):
    ###  aqui realiza 10 epocas
    for j in range(0,Nfilas):
        # el indice J recorre las filas del dataset
        out = Decimal(0.0)
        for i in range(n+1):
            # el indice i recorre las columnas del dataset
            if i == 0 :
                out = Decimal(B[0])
            else:
                out = out + Decimal(B[i]) * Decimal(X[j,i-1])
        pred =Decimal(0.0)
        pred = Decimal(1/(1 + Decimal.exp(-out))) 
        for i in range(n+1):
            
            if i == 0:
                B[0] = Decimal(B[0]) + Decimal(alfa) * (Decimal(Y[j]) - Decimal(pred)) * Decimal(1.0)
            else:
                B[i] = Decimal(B[i]) + Decimal(alfa) * (Decimal(Y[j]) - Decimal(pred)) * Decimal(X[j, i-1])
        
        
            

# Resultado de Los coeficientes B

In [16]:
for i in range(n+1):
   
    print("B[",i,"]",f"{B[i]:10.7f}")

B[ 0 ]  5.8835963
B[ 1 ] -1.3548687
B[ 2 ] -4.5232852
B[ 3 ]  1.6735779
B[ 4 ]  1.3386897
B[ 5 ] -1.6918177
B[ 6 ] -3.3658753
B[ 7 ]  1.3779207
B[ 8 ]  2.0801678
B[ 9 ]  0.7187427
B[ 10 ] -0.3070036
B[ 11 ]  0.6678044
B[ 12 ]  1.3796957
B[ 13 ] -4.0253504
B[ 14 ]  0.6495996
B[ 15 ]  0.3318568


# Valida con el dataset de Prueba

# Lectura del dataset de prueba 

In [17]:
ruta_prueba = "C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/OUTPUT_PCA_Prueba.csv"
df_prueba = pd.read_csv(ruta_prueba , header=0)

In [18]:
X1_prueba = df_prueba.iloc[:, :-1]
X_prueba = np.array(X1_prueba)
Y_prueba = np.array(df_prueba["Y"])

In [19]:
X1_prueba.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
0,-3.175505,-0.356172,-2.727466,1.059857,-0.586492,-2.116135,-1.480299,0.515022,-0.909234,-0.957932,1.453824,0.922110,0.664705,0.909685,2.054646
1,-2.762398,-0.614415,-3.066435,-0.451411,0.052459,-1.487008,0.344435,0.782280,-0.499804,0.445688,0.032907,-1.720888,-0.554592,2.518422,2.746960
2,-3.430038,0.898636,-2.046423,-0.645356,-0.525799,-0.953910,-0.607087,0.397754,-0.029931,-1.419366,2.125340,-0.421915,-0.096791,0.022827,0.601712
3,-0.843711,4.556251,-0.081123,-0.336002,-1.198396,0.000225,-0.077929,-0.423883,-0.333479,0.038871,-2.125210,0.542576,1.458656,-0.555420,0.398301
4,-4.328968,0.416409,-1.344379,-0.308879,-0.213435,-1.101696,-0.506488,2.463260,-4.114533,2.378199,2.290988,0.044182,-0.071782,-0.480098,0.805289


In [20]:

np = X_prueba.shape[1]
Nfilasp = X_prueba.shape[0]
PYE = pd.Series([0] * Nfilasp)
YE = pd.Series([0] * Nfilasp)

# Evaluación de la función logística con los parametros de aprendizaje

In [21]:
for j in range(0,Nfilasp):
        out=0
        for i in range(np+1):
                if i == 0:
                        out = Decimal(out) + Decimal(B[0])*Decimal(1)
                       
                else:
                        out = Decimal(out) + Decimal(B[i]) * Decimal(X_prueba[j,i-1])
                       
        pred = 1/(1 + Decimal.exp(-out))
        
        PYE[j] = pred
        if pred > 0.5 :
                YE[j]=1
        else:
                YE[j]=0


In [22]:
Aciertos = 0
Desaciertos = 0
for j in range(0,Nfilasp):
    if int(Y_prueba[j]) == int(YE[j]):
        Aciertos =Aciertos + 1
        #print(Y_prueba[j], YE[j],PYE[j])
    else:
        Desaciertos = Desaciertos + 1
        #print(Y_prueba[j], YE[j],PYE[j])
print("===================================================================")
print ("Aciertos : ", Aciertos,"Desaciertos" ,Desaciertos)
print ("Exactitud = Aciertos /( Aciertos + Desaciertos) = ", (Aciertos /(Aciertos + Desaciertos))*100,"%")  
print("===================================================================")  

Aciertos :  400 Desaciertos 0
Exactitud = Aciertos /( Aciertos + Desaciertos) =  100.0 %
